In [1]:
import torchvision
from torchvision import transforms
import torch
from torch import nn
from torch.utils.data import DataLoader


In [2]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


In [15]:
device = "cuda"
# Cargar los estados de los modelos
estado_fase_0 = torch.load('Fase_0.pth')
estado_fase_1 = torch.load('Fase_1.pth')


In [16]:
print(estado_fase_1["linear_relu_stack.4.weight"])
print(estado_fase_1["linear_relu_stack.4.bias"])

tensor([[ 0.0383, -0.0326,  0.0727,  ...,  0.0039, -0.0428, -0.0378],
        [ 0.0135, -0.0440,  0.0140,  ..., -0.0180,  0.0154, -0.0127],
        [-0.0364,  0.0376,  0.0563,  ..., -0.0163,  0.0076,  0.0382],
        ...,
        [ 0.0009,  0.0627, -0.0303,  ..., -0.0268, -0.0165,  0.0440],
        [-0.0027, -0.0010,  0.0692,  ..., -0.0151, -0.0139, -0.0044],
        [ 0.0411,  0.0322, -0.0752,  ..., -0.0412, -0.0216, -0.0188]],
       device='cuda:0')
tensor([-0.0321,  0.1792, -0.0126, -0.0387,  0.0772,  0.0711, -0.0166,  0.0924,
        -0.1790, -0.0467], device='cuda:0')


In [17]:

inicio, fin = 5, 10
# Transferir el clasificador de Fase_0 a Fase_1
estado_fase_1["linear_relu_stack.4.weight"][inicio:fin, :] = estado_fase_0["linear_relu_stack.4.weight"][inicio:fin, :]
estado_fase_1["linear_relu_stack.4.bias"][inicio:fin] = estado_fase_0["linear_relu_stack.4.bias"][inicio:fin]


In [18]:

# Crear el modelo Fase_1 y cargar sus estados actualizados
model_classifier0_features1 = NeuralNetwork().to(device)
model_classifier0_features1.load_state_dict(estado_fase_1)



<All keys matched successfully>

### Tunieando el ultimo clasificador

In [56]:
completo = torch.load("Completo.pth")
estado_fase_1 = torch.load('Fase_1.pth')

In [57]:

inicio, fin = 0, 10
# Transferir el clasificador de Fase_0 a Fase_1
estado_fase_1["linear_relu_stack.4.weight"][inicio:fin, :] = completo["linear_relu_stack.4.weight"][inicio:fin, :]
estado_fase_1["linear_relu_stack.4.bias"][inicio:fin] = completo["linear_relu_stack.4.bias"][inicio:fin]


In [58]:
# Crear el modelo Fase_1 y cargar sus estados actualizado
model_last_tune = NeuralNetwork().to(device)
model_last_tune.load_state_dict(estado_fase_1)

<All keys matched successfully>

### Tunieando el primer clasificador

In [63]:
completo = torch.load("Completo.pth")
estado_fase_0 = torch.load('Fase_0.pth')

In [64]:
inicio, fin = 0, 10
# Transferir el clasificador de Fase_0 a Fase_1
estado_fase_0["linear_relu_stack.4.weight"][inicio:fin, :] = completo["linear_relu_stack.4.weight"][inicio:fin, :]
estado_fase_0["linear_relu_stack.4.bias"][inicio:fin] = completo["linear_relu_stack.4.bias"][inicio:fin]


In [65]:
# Crear el modelo Fase_1 y cargar sus estados actualizado
model_first_tune = NeuralNetwork().to(device)
model_first_tune.load_state_dict(estado_fase_0)

<All keys matched successfully>

In [66]:
def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    tarjet_prediction = []  # Lista para almacenar las etiquetas reales y predicciones

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

            # Guardar la etiqueta real y la predicción en la lista de tuplas
            tarjet_prediction.extend(list(zip(y.cpu().numpy(), pred.argmax(1).cpu().numpy())))

    test_loss /= num_batches
    correct /= size
    log_accuracy_loss.append((100*correct, test_loss))
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

    with open(f'tune_first_classifier.txt', 'w') as archivo:
            # Escribe el valor de la variable en el archivo
            archivo.write(str(tarjet_prediction))
    print(f'El valor prediciones se ha guardado en el archivo.txt')
    

In [67]:
eval_data = torchvision.datasets.MNIST(
    root="../data",
    train=False,
    download=True,
    transform=transforms.ToTensor()
)
eval_dataloader = DataLoader(eval_data, batch_size=10000, shuffle=True)
print("Se cargaron los datos correctamente")

Se cargaron los datos correctamente


In [68]:
learning_rate = 1e-3
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model_first_tune.parameters(), lr=learning_rate)
epochs = 1
log_accuracy_loss = []
test_loop(eval_dataloader, model_first_tune, loss_fn)


Test Error: 
 Accuracy: 7.3%, Avg loss: 2.539420 

El valor prediciones se ha guardado en el archivo.txt
